In [1]:
import csv
import pandas as pd

In [2]:
# Note: us_foreign_aid_complete.csv is to large to add to github
usaid_df = pd.read_csv('data/us_foreign_aid_complete.csv')
usaid_df.shape

/Users/chris.marchetti/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1003458, 48)

In [54]:
# Simplifying data to to columns and to disbursements
usaid_df = usaid_df.query("transaction_type_id == 3")
usaid_df = usaid_df[['country_code', 'country_name', 'region_name', 'income_group_id','implementing_agency_acronym', 
                     'implementing_agency_name', 'funding_agency_name','transaction_type_name', 'fiscal_year', 
                     'current_amount','constant_amount']]
usaid_df.head()

,country_code,country_name,region_name,income_group_id,implementing_agency_acronym,implementing_agency_name,funding_agency_name,transaction_type_name,fiscal_year,current_amount,constant_amount
2,AFG,Afghanistan,South and Central Asia,1.0,DOD,Department of Defense,Department of the Army,Disbursements,2011,7840175215,8607968532
3,AFG,Afghanistan,South and Central Asia,1.0,DOD,Department of Defense,Department of the Army,Disbursements,2013,7764310985,8232733951
5,AFG,Afghanistan,South and Central Asia,1.0,DOD,Department of Defense,Department of the Army,Disbursements,2015,6775000000,6971162251
6,AFG,Afghanistan,South and Central Asia,1.0,DOD,Department of Defense,Department of the Army,Disbursements,2012,6642737870,7162444487
8,AFG,Afghanistan,South and Central Asia,1.0,DOD,Department of Defense,Department of the Army,Disbursements,2008,6255388125,7151455862


In [76]:
usaid_df.shape

(506346, 11)

In [55]:
#Limiting data to top 50 countries or regions that recieved US Aid
top_50_df = usaid_df.groupby('country_name').agg({
    'current_amount': 'sum',
    'constant_amount':'sum'
})
top_50_df.sort_values(by = ['current_amount'], inplace = True, ascending = False)
list_of_top_50 = list(top_50_df.index[:50])
new_aid_df = usaid_df[usaid_df.country_name.isin(list_of_top_50)]
new_aid_df.head()


,country_code,country_name,region_name,income_group_id,implementing_agency_acronym,implementing_agency_name,funding_agency_name,transaction_type_name,fiscal_year,current_amount,constant_amount
2,AFG,Afghanistan,South and Central Asia,1.0,DOD,Department of Defense,Department of the Army,Disbursements,2011,7840175215,8607968532
3,AFG,Afghanistan,South and Central Asia,1.0,DOD,Department of Defense,Department of the Army,Disbursements,2013,7764310985,8232733951
5,AFG,Afghanistan,South and Central Asia,1.0,DOD,Department of Defense,Department of the Army,Disbursements,2015,6775000000,6971162251
6,AFG,Afghanistan,South and Central Asia,1.0,DOD,Department of Defense,Department of the Army,Disbursements,2012,6642737870,7162444487
8,AFG,Afghanistan,South and Central Asia,1.0,DOD,Department of Defense,Department of the Army,Disbursements,2008,6255388125,7151455862


In [56]:
# Export cleaned data to a csv for analysis
new_aid_df.to_csv('data/USAidTop50.csv', index = False, header = True)

In [3]:
worldbank_df = pd.read_csv('data/worldbankdata.csv')

In [65]:
# reformat 'countryname' and 'boardapprovaldate' columns
worldbank_df['countryname'] = worldbank_df['countryname'].str.split(";").str[0]
worldbank_df['boardapprovaldate'] = worldbank_df['boardapprovaldate'].str.split("-").str[0]
worldbank_df['totalamt'] = worldbank_df['totalamt'].str.replace(',', '')
worldbank_df['totalamt'] = worldbank_df['totalamt'].astype(float)


In [67]:
# Consolidate worldbank dataframe to include only the columns of interest and display dataframe
worldbank_df = worldbank_df[['id','regionname','countryname','prodline','lendinginstr','productlinetype','status','project_name','boardapprovaldate','totalamt']]


,id,regionname,countryname,prodline,lendinginstr,productlinetype,status,project_name,boardapprovaldate,totalamt
0,P168940,Africa,Republic of Mozambique,PE,Investment Project Financing,L,Active,Additional Financing to the Agriculture and Na...,2019,60000000.0
1,P165873,East Asia and Pacific,Samoa,PE,Investment Project Financing,L,Active,Samoa Agriculture & Fisheries Productivity and...,2019,19950000.0
2,P166013,Africa,Union of the Comoros,PE,Investment Project Financing,L,Active,Comprehensive Approach to Health System Streng...,2019,30000000.0
3,P166539,Africa,Republic of Ghana,PE,Investment Project Financing,L,Active,Ghana Economic Transformation Project,2019,200000000.0
4,P168414,Middle East and North Africa,Arab Republic of Egypt,PE,Investment Project Financing,L,Active,Strengthening Social Safety Net Additional Fin...,2019,500000000.0


In [68]:
# Function to make country names match the US Data's country names
import re
def country_standard(row):
    try:
        row = str(row)
        # calls regex on that and returns
        row = re.sub(r"The|of|Republic|Kingdom|Commonwealth|Islamic|People's?", " ", row).strip()
        row = re.sub(r"\s{2,}", " ", row)
        return row
    except ValueError:
        print(row)

In [74]:
# Country names fixed
worldbank_df['countryname2']= worldbank_df['countryname'].map(country_standard)


In [75]:
# Taking a subset of the World Bank data where the country names match
worldbank50 = worldbank_df[worldbank_df.countryname2.isin(list_of_top_50)]


In [73]:
# Exporting the World Bank data as a csv with the columns and countries we were going to analyze

worldbank50.to_csv("data/worldbank50.csv",index=False)